In [1]:
!pip install matplotlib
import os
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql import functions as f
from pyspark.sql.types import StringType
import pandas as pd
# import matplotlib.pyplot as plt

# set local variables


%env MINIO_PASSWORD 6HgSzdwj8eNpHcux
%env MINIO_USERNAME grupo-02

  Using cached https://files.pythonhosted.org/packages/b7/65/d6e00376dbdb6c227d79a2d6ec32f66cfb163f0cd924090e3133a4f85a11/matplotlib-3.7.1.tar.gz
  Installing build dependencies ... done
  Using cached https://files.pythonhosted.org/packages/b4/9b/6edb9d3e334a70a212f66a844188fcb57ddbd528cbc3b1fe7abfc317ddd7/contourpy-1.0.7.tar.gz
  Installing build dependencies ... done
  Using cached https://files.pythonhosted.org/packages/5c/f9/695d6bedebd747e5eb0fe8fad57b72fdf25411273a39791cde838d5a8f51/cycler-0.11.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e3/d9/e9bae85e84737e76ebbcbea13607236da0c0699baed0ae4f1151b728a608/fonttools-4.38.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/86/7a/6b438da7534dacd232ed4e19f74f4edced2cda9494d7e6536f54edfdf4a5/kiwisolver-1.4.4-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/bc/07/830784e061fb94d67649f3e438ff63cfb902dec6d48ac75aeaaac7c7c30e/Pill

In [2]:
import os
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql import functions as f
from pyspark.sql.types import StringType


os.environ[
    "PYSPARK_SUBMIT_ARGS"
] = "--packages org.apache.hadoop:hadoop-aws:3.2.2,io.delta:delta-core_2.12:1.1.0  pyspark-shell "

os.environ['MINIO_USERNAME'] = 'grupo-02'
os.environ['MINIO_PASSWORD'] = '6HgSzdwj8eNpHcux'
        
config = {
    "spark.jars.packages":"org.apache.hadoop:hadoop-aws:3.2.2",
    "spark.kubernetes.namespace": "spark",
    "spark.kubernetes.container.image": "cronosnull/abd-spark-base:202301",
    "spark.executor.instances": "20",
    "spark.executor.memory": "10g",
    "spark.executor.cores": "1",
    "spark.driver.memory":"3g",
    "spark.driver.port":"38890",
    "spark.driver.blockManager.port":"7778",
    "spark.driver.bindAddress": "0.0.0.0",
    "spark.driver.host": "172.24.99.147",
    "spark.kubernetes.executor.request.cores":"500m",
    "spark.hadoop.fs.s3a.endpoint": "http://172.24.99.18:9000",
    # Credenciales de MinNIO, no olvide asignar las variables de entorno  
    "spark.hadoop.fs.s3a.access.key": os.environ.get('MINIO_USERNAME', "--"),
    "spark.hadoop.fs.s3a.secret.key": os.environ.get('MINIO_PASSWORD', "--"),
    "spark.hadoop.fs.s3a.path.style.access": True,
    "spark.hadoop.fs.s3a.impl": "org.apache.hadoop.fs.s3a.S3AFileSystem",
    "spark.hadoop.fs.s3a.aws.credentials.provider": "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider",
    "spark.kubernetes.local.dirs.tmpfs":True,

}

def get_spark_session(app_name: str, conf: SparkConf):
    conf.setMaster("k8s://https://172.24.99.68:16443")
    for key, value in config.items():
        conf.set(key, value)
    conf.set("spark.ui.port","4042")
    return SparkSession.builder.appName(app_name).config(conf=conf).getOrCreate()

In [3]:
spark = get_spark_session("tarea-1-spark", SparkConf())

In [4]:
!spark-submit --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.2.0
      /_/
                        
Using Scala version 2.12.15, OpenJDK 64-Bit Server VM, 1.8.0_322
Branch 
Compiled by user  on 2022-03-26T09:34:47Z
Revision 
Url 
Type --help for more information.


In [5]:
spark

# Parte 2


In [6]:
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import col, when
from pyspark.sql.functions import to_date
from pyspark.sql.functions import date_format


# Quitar espacios en los nombres es las columnas
def remove_space_in_names(df):
    for col_name in df.columns:
        new_col_name = col_name.replace(' ', '')
        df = df.withColumnRenamed(col_name, new_col_name)
    return df


# Cambiar los tipos de datos del dataframe
# Falta Rate_Code, store_and_forward, mta_tax
def change_name_and_data_type_G1(df):
    
    df = remove_space_in_names(df)

    df = df.withColumnRenamed("vendor_name", "vendor_id").withColumn("vendor_id", col("vendor_id").cast("string"))
    df = df.withColumnRenamed("Trip_Pickup_DateTime", "pickup_datetime").withColumn("pickup_datetime", to_timestamp("pickup_datetime", "yyyy-MM-dd HH:mm:ss"))
    df = df.withColumnRenamed("Trip_Dropoff_DateTime", "dropoff_datetime").withColumn("dropoff_datetime", to_timestamp("dropoff_datetime", "yyyy-MM-dd HH:mm:ss"))
    df = df.withColumnRenamed("Passenger_Count", "passenger_count").withColumn("passenger_count", col("passenger_count").cast("integer"))
    df = df.withColumnRenamed("Trip_Distance", "trip_distance").withColumn("trip_distance", col("trip_distance").cast("double"))
    df = df.withColumnRenamed("Start_Lon", "pickup_longitude").withColumn("pickup_longitude", col("pickup_longitude").cast("double"))
    df = df.withColumnRenamed("Start_Lat", "pickup_latitude").withColumn("pickup_latitude", col("pickup_latitude").cast("double"))
    df = df.withColumnRenamed("Rate_Code", "rate_code").withColumn("rate_code", col("rate_code").cast("string"))
    df = df.withColumnRenamed("store_and_forward", "store_and_fwd_flag").withColumn("store_and_fwd_flag", col("store_and_fwd_flag").cast("string"))
    df = df.withColumnRenamed("End_Lon", "dropoff_longitude").withColumn("dropoff_longitude", col("dropoff_longitude").cast("double"))
    df = df.withColumnRenamed("End_Lat", "dropoff_latitude").withColumn("dropoff_latitude", col("dropoff_latitude").cast("double"))
    df = df.withColumnRenamed("Payment_Type", "payment_type").withColumn("payment_type", col("payment_type").cast("string"))
    df = df.withColumnRenamed("Fare_Amt", "fare_amount").withColumn("fare_amount", col("fare_amount").cast("double"))
    df = df.withColumnRenamed("surcharge", "surcharge").withColumn("surcharge", col("surcharge").cast("double"))
    df = df.withColumnRenamed("mta_tax", "mta_tax").withColumn("mta_tax", col("mta_tax").cast("string"))
    df = df.withColumnRenamed("Tip_Amt", "tip_amount").withColumn("tip_amount", col("tip_amount").cast("double"))
    df = df.withColumnRenamed("Tolls_Amt", "tolls_amount").withColumn("tolls_amount", col("tolls_amount").cast("double"))
    df = df.withColumnRenamed("Total_Amt", "total_amount").withColumn("total_amount", col("total_amount").cast("double"))
    
    #Crear nueva columna
    df= df.withColumn("date_travel", date_format("pickup_datetime", "yyyy-MM-dd").alias("date_travel"))
    df = df.withColumn("date_travel", to_date("date_travel", "yyyy-MM-dd"))
    
    return df

def change_name_and_data_type_G2(df):
    
    df = remove_space_in_names(df)

    df = df.withColumnRenamed("vendor_id", "vendor_id").withColumn("vendor_id", col("vendor_id").cast("string"))
    df = df.withColumnRenamed("pickup_datetime", "pickup_datetime").withColumn("pickup_datetime", to_timestamp("pickup_datetime", "yyyy-MM-dd HH:mm:ss"))
    df = df.withColumnRenamed("dropoff_datetime", "dropoff_datetime").withColumn("dropoff_datetime", to_timestamp("dropoff_datetime", "yyyy-MM-dd HH:mm:ss"))
    df = df.withColumnRenamed("passenger_count", "passenger_count").withColumn("passenger_count", col("passenger_count").cast("integer"))
    df = df.withColumnRenamed("trip_distance", "trip_distance").withColumn("trip_distance", col("trip_distance").cast("double"))
    df = df.withColumnRenamed("pickup_longitude", "pickup_longitude").withColumn("pickup_longitude", col("pickup_longitude").cast("double"))
    df = df.withColumnRenamed("pickup_latitude", "pickup_latitude").withColumn("pickup_latitude", col("pickup_latitude").cast("double"))
    df = df.withColumnRenamed("rate_code", "rate_code").withColumn("rate_code", col("rate_code").cast("string"))
    df = df.withColumnRenamed("store_and_fwd_flag", "store_and_fwd_flag").withColumn("store_and_fwd_flag", col("store_and_fwd_flag").cast("string"))
    df = df.withColumnRenamed("dropoff_longitude", "dropoff_longitude").withColumn("dropoff_longitude", col("dropoff_longitude").cast("double"))
    df = df.withColumnRenamed("dropoff_latitude", "dropoff_latitude").withColumn("dropoff_latitude", col("dropoff_latitude").cast("double"))
    df = df.withColumnRenamed("payment_type", "payment_type").withColumn("payment_type", col("payment_type").cast("string"))
    df = df.withColumnRenamed("fare_amount", "fare_amount").withColumn("fare_amount", col("fare_amount").cast("double"))
    df = df.withColumnRenamed("surcharge", "surcharge").withColumn("surcharge", col("surcharge").cast("double"))
    df = df.withColumnRenamed("mta_tax", "mta_tax").withColumn("mta_tax", col("mta_tax").cast("string"))
    df = df.withColumnRenamed("tip_amount", "tip_amount").withColumn("tip_amount", col("tip_amount").cast("double"))
    df = df.withColumnRenamed("tolls_amount", "tolls_amount").withColumn("tolls_amount", col("tolls_amount").cast("double"))
    df = df.withColumnRenamed("total_amount", "total_amount").withColumn("total_amount", col("total_amount").cast("double"))
    
   
#     #Crear nueva columna
    df= df.withColumn("date_travel", date_format("pickup_datetime", "yyyy-MM-dd").alias("date_travel"))
    df = df.withColumn("date_travel", to_date("date_travel", "yyyy-MM-dd"))
    
    return df


def change_name_and_data_type_G3(df):
    
    df = remove_space_in_names(df)

    df = df.withColumnRenamed("VendorID", "vendor_id").withColumn("vendor_id", col("vendor_id").cast("string"))
    df = df.withColumnRenamed("tpep_pickup_datetime", "pickup_datetime").withColumn("pickup_datetime", to_timestamp("pickup_datetime", "yyyy-MM-dd HH:mm:ss"))
    df = df.withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime").withColumn("dropoff_datetime", to_timestamp("dropoff_datetime", "yyyy-MM-dd HH:mm:ss"))
    df = df.withColumnRenamed("passenger_count", "passenger_count").withColumn("passenger_count", col("passenger_count").cast("integer"))
    df = df.withColumnRenamed("trip_distance", "trip_distance").withColumn("trip_distance", col("trip_distance").cast("double"))
    df = df.withColumnRenamed("pickup_longitude", "pickup_longitude").withColumn("pickup_longitude", col("pickup_longitude").cast("double"))
    df = df.withColumnRenamed("pickup_latitude", "pickup_latitude").withColumn("pickup_latitude", col("pickup_latitude").cast("double"))
    df = df.withColumnRenamed("RateCodeID", "rate_code").withColumn("rate_code", col("rate_code").cast("string"))
    df = df.withColumnRenamed("store_and_fwd_flag", "store_and_fwd_flag").withColumn("store_and_fwd_flag", col("store_and_fwd_flag").cast("string"))
    df = df.withColumnRenamed("dropoff_longitude", "dropoff_longitude").withColumn("dropoff_longitude", col("dropoff_longitude").cast("double"))
    df = df.withColumnRenamed("dropoff_latitude", "dropoff_latitude").withColumn("dropoff_latitude", col("dropoff_latitude").cast("double"))
    df = df.withColumnRenamed("payment_type", "payment_type").withColumn("payment_type", col("payment_type").cast("string"))
    df = df.withColumnRenamed("fare_amount", "fare_amount").withColumn("fare_amount", col("fare_amount").cast("double"))
    df = df.withColumnRenamed("improvement_surcharge", "surcharge").withColumn("surcharge", col("surcharge").cast("double"))
    df = df.withColumnRenamed("mta_tax", "mta_tax").withColumn("mta_tax", col("mta_tax").cast("string"))
    df = df.withColumnRenamed("tip_amount", "tip_amount").withColumn("tip_amount", col("tip_amount").cast("double"))
    df = df.withColumnRenamed("tolls_amount", "tolls_amount").withColumn("tolls_amount", col("tolls_amount").cast("double"))
    df = df.withColumnRenamed("total_amount", "total_amount").withColumn("total_amount", col("total_amount").cast("double"))
   
    #Eliminar columna no usada
    df = df.drop("extra")
    
   
    #Crear nueva columna
    df= df.withColumn("date_travel", date_format("pickup_datetime", "yyyy-MM-dd").alias("date_travel"))
    df = df.withColumn("date_travel", to_date("date_travel", "yyyy-MM-dd"))
    
    return df


def change_name_and_data_type_G4(df):
    
    df = remove_space_in_names(df)

    df = df.withColumnRenamed("VendorID", "vendor_id").withColumn("vendor_id", col("vendor_id").cast("string"))
    df = df.withColumnRenamed("tpep_pickup_datetime", "pickup_datetime").withColumn("pickup_datetime", to_timestamp("pickup_datetime", "yyyy-MM-dd HH:mm:ss"))
    df = df.withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime").withColumn("dropoff_datetime", to_timestamp("dropoff_datetime", "yyyy-MM-dd HH:mm:ss"))
    df = df.withColumnRenamed("passenger_count", "passenger_count").withColumn("passenger_count", col("passenger_count").cast("integer"))
    df = df.withColumnRenamed("trip_distance", "trip_distance").withColumn("trip_distance", col("trip_distance").cast("double"))
    df = df.withColumnRenamed("RateCodeID", "rate_code").withColumn("rate_code", col("rate_code").cast("string"))
    df = df.withColumnRenamed("store_and_fwd_flag", "store_and_fwd_flag").withColumn("store_and_fwd_flag", col("store_and_fwd_flag").cast("string"))
    df = df.withColumn("PULocationID", col("PULocationID").cast("integer"))
    df = df.withColumn("PULocationID", col("PULocationID").cast("integer"))
    
#     df = df.withColumnRenamed("PULocationID", "pickup_zone").withColumn("pickup_zone", col("pickup_zone").cast("string"))
#     df = df.withColumnRenamed("PULocationID", "dropoff_zone").withColumn("pickup_zone", col("pickup_zone").cast("string"))

    df = df.withColumnRenamed("payment_type", "payment_type").withColumn("payment_type", col("payment_type").cast("string"))
    df = df.withColumnRenamed("fare_amount", "fare_amount").withColumn("fare_amount", col("fare_amount").cast("double"))
    df = df.withColumnRenamed("improvement_surcharge", "surcharge").withColumn("surcharge", col("surcharge").cast("double"))
    df = df.withColumnRenamed("mta_tax", "mta_tax").withColumn("mta_tax", col("mta_tax").cast("string"))
    df = df.withColumnRenamed("tip_amount", "tip_amount").withColumn("tip_amount", col("tip_amount").cast("double"))
    df = df.withColumnRenamed("tolls_amount", "tolls_amount").withColumn("tolls_amount", col("tolls_amount").cast("double"))
    df = df.withColumnRenamed("total_amount", "total_amount").withColumn("total_amount", col("total_amount").cast("double"))
    
    #Eliminar columna no usada
    df = df.drop("extra")
    
    #Crear nueva columna
    df= df.withColumn("date_travel", date_format("pickup_datetime", "yyyy-MM-dd").alias("date_travel"))
    df = df.withColumn("date_travel", to_date("date_travel", "yyyy-MM-dd"))
    
    return df


In [7]:
# charge info group 1
df_taxis_G1 = spark.read.csv("s3a://taxis/yellow_tripdata_2009-*.csv.gz", header=True)
df_taxis_G1_change_type = change_name_and_data_type_G1(df_taxis_G1)

df_taxis_G1_change_type.printSchema()

root
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- surcharge: double (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- date_travel: date (nullable = true)



In [8]:
# charge info group 2

df_taxis_G2_2010 = spark.read.csv("s3a://taxis/yellow_tripdata_2010-*.csv.gz", header=True)
df_taxis_G2_2010_change_type = change_name_and_data_type_G2(df_taxis_G2_2010)

df_taxis_G2_2011 = spark.read.csv("s3a://taxis/yellow_tripdata_2011-*.csv.gz", header=True)
df_taxis_G2_2011_change_type = change_name_and_data_type_G2(df_taxis_G2_2011)

df_taxis_G2_2012 = spark.read.csv("s3a://taxis/yellow_tripdata_2012-*.csv.gz", header=True)
df_taxis_G2_2012_change_type = change_name_and_data_type_G2(df_taxis_G2_2012)

df_taxis_G2_2013 = spark.read.csv("s3a://taxis/yellow_tripdata_2013-*.csv.gz", header=True)
df_taxis_G2_2013_change_type = change_name_and_data_type_G2(df_taxis_G2_2013)

df_taxis_G2_2014 = spark.read.csv("s3a://taxis/yellow_tripdata_2014-*.csv.gz", header=True)
df_taxis_G2_2014_change_type = change_name_and_data_type_G2(df_taxis_G2_2014)


df_taxis_G2_change_type = df_taxis_G2_2010_change_type.unionAll(df_taxis_G2_2011_change_type).unionAll(df_taxis_G2_2013_change_type).unionAll(df_taxis_G2_2014_change_type)

df_taxis_G1_change_type.printSchema()


root
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- surcharge: double (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- date_travel: date (nullable = true)



In [9]:
# charge info group 3

df_taxis_G3_2015 = spark.read.csv("s3a://taxis/yellow_tripdata_2015-*.csv.gz", header=True)
df_taxis_G3_2015_change_type = change_name_and_data_type_G3(df_taxis_G3_2015)

df_taxis_G3_2016 = spark.read.csv("s3a://taxis/yellow_tripdata_2016-*.csv.gz", header=True)
df_taxis_G3_2016_change_type = change_name_and_data_type_G3(df_taxis_G3_2016)

df_taxis_G3_change_type = df_taxis_G3_2015_change_type.unionAll(df_taxis_G3_2016_change_type)

df_taxis_G3_change_type.printSchema()

root
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- date_travel: date (nullable = true)



In [10]:
# charge info group 4

df_taxis_G4_2017 = spark.read.csv("s3a://taxis/yellow_tripdata_2017-*.csv.gz", header=True)
df_taxis_G4_2017_change_type = change_name_and_data_type_G4(df_taxis_G4_2017)

df_taxis_G4_2018 = spark.read.csv("s3a://taxis/yellow_tripdata_2018-*.csv.gz", header=True)
df_taxis_G4_2018_change_type = change_name_and_data_type_G4(df_taxis_G4_2018)

df_taxis_G4_2019 = spark.read.csv("s3a://taxis/yellow_tripdata_2019-*.csv.gz", header=True)
df_taxis_G4_2019 = df_taxis_G4_2019.drop("congestion_surcharge")
df_taxis_G4_2019_change_type = change_name_and_data_type_G4(df_taxis_G4_2019)


df_taxis_G4_change_type = df_taxis_G4_2017_change_type.unionAll(df_taxis_G4_2018_change_type).unionAll(df_taxis_G4_2019_change_type)

df_taxis_G4_change_type.printSchema()



root
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- date_travel: date (nullable = true)



In [11]:
# Add the zone name
df_taxis_G4_change_type_2 = df_taxis_G4_change_type

df_location_id = spark.read.csv("s3a://user-data/grupo-02/taxi+_zone_lookup.csv", header=True)

df_location_id.head(4)

[Row(LocationID='1', Borough='EWR', Zone='Newark Airport', service_zone='EWR'),
 Row(LocationID='2', Borough='Queens', Zone='Jamaica Bay', service_zone='Boro Zone'),
 Row(LocationID='3', Borough='Bronx', Zone='Allerton/Pelham Gardens', service_zone='Boro Zone'),
 Row(LocationID='4', Borough='Manhattan', Zone='Alphabet City', service_zone='Yellow Zone')]

In [12]:
# Add the zone to the dataframe 4 
df_taxis_G4_change_type_2 = df_taxis_G4_change_type

df_taxis_G4_change_type_2 = df_taxis_G4_change_type_2.join(df_location_id, df_taxis_G4_change_type_2.PULocationID == df_location_id.LocationID).select(df_taxis_G4_change_type_2['*'], df_location_id['Zone'])
df_taxis_G4_change_type_2 = df_taxis_G4_change_type_2.withColumnRenamed("Zone", "pickup_zone")

df_taxis_G4_change_type_2 = df_taxis_G4_change_type_2.alias('a').join(df_location_id.alias('b'), df_taxis_G4_change_type_2.alias('a').DOLocationID == df_location_id.alias('b').LocationID).select('a.*','b.Zone')
df_taxis_G4_change_type_2 = df_taxis_G4_change_type_2.withColumnRenamed("Zone", "dropoff_zone")
    
df_taxis_G4_change_type_2 = df_taxis_G4_change_type_2.drop('DOLocationID','PULocationID')
df_taxis_G4_change_type_2.printSchema()



root
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- date_travel: date (nullable = true)
 |-- pickup_zone: string (nullable = true)
 |-- dropoff_zone: string (nullable = true)



In [13]:
# Join all the data with latitude and longitud

df_taxis_with_lat_lon = df_taxis_G1_change_type.unionAll(df_taxis_G2_change_type).unionAll(df_taxis_G3_change_type)

df_taxis_with_lat_lon.printSchema()

root
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- surcharge: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- date_travel: date (nullable = true)



## Geo pandas


In [14]:
# !pip install geopandas

In [16]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, shape
from shapely import wkb, wkt
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, IntegerType, FloatType, DoubleType,DecimalType
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf

import shapely.speedups
shapely.speedups.enable() # this makes some spatial queries run faster

#Load the data 
df_taxi_zones_csv = spark.read.csv("s3a://user-data/grupo-02/taxi_zones.csv", header=True)
df_taxi_zones_csv.printSchema()

# define a function to convert a WKT string to a Shapely geometry object
def wkt_to_geometry(wkt_str):
    return wkt.loads(wkt_str)

# read in the CSV file and convert the 'the_geom' column to a geometry data type
df_taxi_zones_csv = pd.read_csv("taxi_zones.csv", dtype={'the_geom': 'str'}).assign(the_geom=lambda x: x['the_geom'].apply(wkt_to_geometry))

# print the data types of the columns
print(df_taxi_zones_csv.dtypes)

gdf  = gpd.GeoDataFrame(df_taxi_zones_csv, geometry='the_geom')

# spark.broadcast(gdf)

def find_zone(latitude, longitude): 
    mgdf = gdf.apply(lambda x: x['zone'] if x['the_geom'].intersects(Point(longitude,latitude)) else None, axis=1)
    idx = mgdf.first_valid_index()
    first_valid_value = mgdf.loc[idx] if idx is not None else None
    return first_valid_value



find_zone_udf = udf(lambda lat, lon: find_zone(lat, lon), StringType())




root
 |-- OBJECTID: string (nullable = true)
 |-- Shape_Leng: string (nullable = true)
 |-- the_geom: string (nullable = true)
 |-- Shape_Area: string (nullable = true)
 |-- zone: string (nullable = true)
 |-- LocationID: string (nullable = true)
 |-- borough: string (nullable = true)

OBJECTID        int64
Shape_Leng    float64
the_geom       object
Shape_Area    float64
zone           object
LocationID      int64
borough        object
dtype: object


In [17]:

df_taxis_with_lat_lon= df_taxis_with_lat_lon.withColumn("pickup_zone", find_zone_udf(col("pickup_latitude"),col("pickup_longitude")))

df_taxis_with_lat_lon= df_taxis_with_lat_lon.withColumn("dropoff_zone", find_zone_udf(col("dropoff_latitude"),col("dropoff_longitude")))

df_taxis_with_lat_lon = df_taxis_with_lat_lon.drop('pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude')
df_taxis_with_lat_lon.printSchema()


root
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- surcharge: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- date_travel: date (nullable = true)
 |-- pickup_zone: string (nullable = true)
 |-- dropoff_zone: string (nullable = true)



In [18]:
# Join all the data

final_df_taxi_zones  = df_taxis_with_lat_lon.unionAll(df_taxis_G4_change_type_2)

final_df_taxi_zones= final_df_taxi_zones.withColumn("year_travel", date_format("pickup_datetime", "yyyy").alias("year_travel"))
final_df_taxi_zones= final_df_taxi_zones.withColumn("month_travel", date_format("pickup_datetime", "MM").alias("month_travel"))
final_df_taxi_zones= final_df_taxi_zones.withColumn("day_travel", date_format("pickup_datetime", "dd").alias("day_travel"))


final_df_taxi_zones = final_df_taxi_zones.withColumn("year_travel", col("year_travel").cast("integer"))
final_df_taxi_zones = final_df_taxi_zones.withColumn("month_travel", col("year_travel").cast("integer"))
final_df_taxi_zones = final_df_taxi_zones.withColumn("day_travel", col("year_travel").cast("integer"))


final_df_taxi_zones.printSchema()


root
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- surcharge: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- date_travel: date (nullable = true)
 |-- pickup_zone: string (nullable = true)
 |-- dropoff_zone: string (nullable = true)
 |-- year_travel: integer (nullable = true)
 |-- month_travel: integer (nullable = true)
 |-- day_travel: integer (nullable = true)



In [19]:
# Se crea un dataframe para analizar los datos

pd_df_final_df_taxi_zones = final_df_taxi_zones.toPandas()


Py4JJavaError: An error occurred while calling o1376.collectToPython.
: org.apache.spark.SparkException: Job 17 cancelled 
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2403)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:2299)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:2550)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2533)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2522)
	at org.apache.spark.util.EventLoop$$anon$1.run(EventLoop.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:898)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2214)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2235)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2254)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2

In [ ]:
# Se sube el CSV a Minio

final_df_taxi_zones.write.mode("overwrite").csv("s3a://user-data/grupo-02/prueba/prueba.csv")


# Analisis de los datos


## Punto G


## Punto H


In [ ]:
most_popular_destinations_per_years = pd_df_final_df_taxi_zones.groupby(['year_travel', 'dropoff_zone']).size().reset_index(name='count')

most_popular_destinations_per_years = most_popular_destinations_per_years.sort_values(['year_travel', 'count'], ascending=[False, False])

most_popular_destinations_per_years


## Punto I

In [ ]:
df_km_per_year = pd_df_final_df_taxi_zones.groupby(['year_travel'])['trip_distance'].sum()

df_km_per_year = df_km_per_year.to_frame().reset_index()

df_km_per_year["trip_distance_km"] = df_km_per_year["trip_distance"] * 1.609344


In [ ]:
plt.plot(df_km_per_year["year_travel"], df_km_per_year["trip_distance_km"], '-o')


plt.xlabel('Año')
plt.xlim(2007, 2020)

plt.ylabel('Distancia total recorrida en Kilometros ')

plt.title(' Distancia total recorrida en Kilometros por año')

plt.show()

## Punto J

In [24]:
most_popular_destination_per_year_per_month = final_df_taxi_zones.groupby(["year_travel", "month_travel", 'dropoff_zone']).count()

In [28]:
from pyspark.sql.functions import count, desc

In [29]:
most_popular_destination_per_year_per_month = most_popular_destination_per_year_per_month.orderBy(['year_travel', 'month_travel', 'count'], ascending=[False, False, False])

In [ ]:
most_popular_destination_per_year_per_month.head(10)

## Punto K

¿Cuál es la tarifa promedio por destino por año?

In [ ]:
most_expensive_destination_per_year = df.groupby(["year_travel", 'dropoff_zone']).agg(mean("total_amount"))

In [ ]:
most_expensive_destination_per_year = most_expensive_destination_per_year.orderBy(['year_travel', 'avg(total_amount)'], ascending=[False, False])

In [ ]:
most_expensive_destination_per_year.head(10)